In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from torchtext import data,datasets

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
import sys

['imdb.csv']


# Explore Dataset

In [82]:
table = pd.read_csv("../input/imdb.csv", error_bad_lines=False)
table.size

b'Skipping line 66: expected 44 fields, saw 46\nSkipping line 111: expected 44 fields, saw 45\nSkipping line 198: expected 44 fields, saw 45\nSkipping line 222: expected 44 fields, saw 46\nSkipping line 278: expected 44 fields, saw 45\nSkipping line 396: expected 44 fields, saw 45\nSkipping line 403: expected 44 fields, saw 45\nSkipping line 421: expected 44 fields, saw 45\nSkipping line 437: expected 44 fields, saw 45\nSkipping line 462: expected 44 fields, saw 46\nSkipping line 491: expected 44 fields, saw 45\nSkipping line 515: expected 44 fields, saw 45\nSkipping line 529: expected 44 fields, saw 45\nSkipping line 530: expected 44 fields, saw 45\nSkipping line 558: expected 44 fields, saw 45\nSkipping line 623: expected 44 fields, saw 45\nSkipping line 646: expected 44 fields, saw 45\nSkipping line 663: expected 44 fields, saw 46\nSkipping line 713: expected 44 fields, saw 45\nSkipping line 730: expected 44 fields, saw 47\nSkipping line 791: expected 44 fields, saw 45\nSkipping lin

630608

## Show columns

In [83]:
table.columns

Index(['fn', 'tid', 'title', 'wordsInTitle', 'url', 'imdbRating',
       'ratingCount', 'duration', 'year', 'type', 'nrOfWins',
       'nrOfNominations', 'nrOfPhotos', 'nrOfNewsArticles', 'nrOfUserReviews',
       'nrOfGenre', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport',
       'TalkShow', 'Thriller', 'War', 'Western'],
      dtype='object')

# Print two entry

In [84]:
table.iloc[0:2]

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,...,0,0,0,0,0,0,0,0,0,0


## First 10 * 10 

In [85]:
table.iloc[0:10, 1:10]

,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type
0,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie
1,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie
2,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie
3,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie
4,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie
5,tt0022100,M (1931),m,http://www.imdb.com/title/tt0022100/,8.5,73726.0,7020.0,1931.0,video.movie
6,tt0025316,Es geschah in einer Nacht (1934),es geschah in einer nacht,http://www.imdb.com/title/tt0025316/,8.3,46503.0,6300.0,1934.0,video.movie
7,tt0027977,Moderne Zeiten (1936),moderne zeiten,http://www.imdb.com/title/tt0027977/,8.6,90847.0,5220.0,1936.0,video.movie
8,tt0031381,Vom Winde verweht (1939),vom winde verweht,http://www.imdb.com/title/tt0031381/,8.2,160414.0,14280.0,1939.0,video.movie
9,tt0031679,Mr. Smith geht nach Washington (1939),mr smith geht nach washington,http://www.imdb.com/title/tt0031679/,8.4,58169.0,7740.0,1939.0,video.movie


# Prepare Dataset

In [86]:
from torchtext import data

Text = data.Field(lower=True, batch_first=True, fix_length=20)
Label = data.Field(sequential=False)

In [87]:
from torchtext import datasets
train, test = datasets.IMDB.splits(Text, Label)

In [88]:
print(len(train))
train.fields

25000


{'text': <torchtext.data.field.Field at 0x7f34bee5cfd0>,
 'label': <torchtext.data.field.Field at 0x7f34c0a80f98>}

In [89]:
vars(train[0])

{'text': ['i',
  'second',
  'the',
  'motion',
  'to',
  'make',
  'this',
  'into',
  'a',
  'movie,',
  'it',
  'would',
  'be',
  'great!!',
  'i',
  'was',
  'also',
  'amazed',
  'at',
  'the',
  'storyline',
  'and',
  'character',
  'build',
  'in',
  'this',
  'game.',
  'i',
  'have',
  'played',
  'it',
  'again',
  'and',
  'again',
  '(over',
  '20',
  'times)',
  'just',
  'to',
  'try',
  'something',
  'different',
  'and',
  'it',
  'gets',
  'more',
  'interesting',
  'every',
  'time.',
  'final',
  'fantasy',
  'eat',
  'your',
  'heart',
  'out!!',
  'this',
  'should',
  'be',
  'made',
  'into',
  'a',
  'movie!!!!!',
  'if',
  'anyone',
  'out',
  'there',
  'wants',
  'some',
  'help',
  'to',
  'start',
  'a',
  'petition',
  'to',
  'have',
  'this',
  'made',
  'into',
  'a',
  'movie,',
  'please',
  'contact',
  'me.',
  'i',
  'would',
  'love',
  'to',
  'help',
  'with',
  'that',
  'project',
  'any',
  'day.',
  'the',
  'graphics',
  'are',
  'great'

# Build vocabulary

In [90]:
from torchtext.vocab import GloVe,FastText,CharNGram
Text.build_vocab(train, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
Label.build_vocab(train,)

# Frequency

In [91]:
Text.vocab.freqs

Counter({'i': 70480,
         'second': 1625,
         'the': 322198,
         'motion': 301,
         'to': 133967,
         'make': 7590,
         'this': 69714,
         'into': 8990,
         'a': 159953,
         'movie,': 3979,
         'it': 65505,
         'would': 12027,
         'be': 25691,
         'great!!': 4,
         'was': 47024,
         'also': 8007,
         'amazed': 167,
         'at': 22731,
         'storyline': 525,
         'and': 158572,
         'character': 5260,
         'build': 283,
         'in': 90527,
         'game.': 151,
         'have': 27344,
         'played': 2238,
         'again': 1781,
         '(over': 8,
         '20': 529,
         'times)': 25,
         'just': 17309,
         'try': 1656,
         'something': 4387,
         'different': 2064,
         'gets': 3117,
         'more': 13170,
         'interesting': 2364,
         'every': 3885,
         'time.': 1931,
         'final': 1233,
         'fantasy': 451,
         'eat': 239,
 

# vectors

In [92]:
Text.vocab.vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7724, -0.1800,  0.2072,  ...,  0.6736,  0.2263, -0.2919],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [93]:
Text.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             'a': 3,
             'and': 4,
             'of': 5,
             'to': 6,
             'is': 7,
             'in': 8,
             'i': 9,
             'this': 10,
             'that': 11,
             'it': 12,
             '/><br': 13,
             'was': 14,
             'as': 15,
             'for': 16,
             'with': 17,
             'but': 18,
             'on': 19,
             'movie': 20,
             'his': 21,
             'are': 22,
             'not': 23,
             'film': 24,
             'you': 25,
             'have': 26,
             'he': 27,
             'be': 28,
             'at': 29,
             'one': 30,
             'by': 31,
             'an': 32,
             'they': 33,
             'from': 34,
             'all': 35,
             'who': 36,
             'like': 37,
             'so': 38,
             '

# Generate batches of vector

In [94]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=128, device='cuda', shuffle=True)

In [95]:
batch = next(iter(train_iter))

In [96]:
batch.text

tensor([[  10,    7,    3,  ...,  715,    8,   44],
        [  10,    7,    3,  ...,   83, 1095, 2645],
        [   2,   88, 2448,  ...,  161,    4,    0],
        ...,
        [2072,    3,   75,  ...,    4,    2, 1558],
        [  10,   24,  397,  ...,   11, 1661,  856],
        [  10,   20,    7,  ...,   12,  102, 5684]], device='cuda:0')

In [97]:
batch.label

tensor([2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2,
        1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1,
        1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2,
        2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1,
        1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1,
        2, 1, 2, 1, 2, 1, 2, 1], device='cuda:0')

# Create model

In [98]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=200):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc = nn.Linear(hidden_size2,3)
        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc(embeds)
        return F.log_softmax(out,dim=-1)

In [99]:
model = EmbNet(len(Text.vocab.stoi),10)
model = model.cuda()

# Training

In [100]:
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [101]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    #print('{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    print("Phase: {}".format(phase),
         "\tLoss is: {}".format(loss),
         "\tAccuray: {}".format(accuracy))
    
    return loss,accuracy

In [102]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

Phase: training 	Loss is: 0.8509522882080078 	Accuray: 49
Phase: validation 	Loss is: 0.7289032999420166 	Accuray: 52
Phase: training 	Loss is: 0.7081740489196777 	Accuray: 54
Phase: validation 	Loss is: 0.7004621440124512 	Accuray: 54
Phase: training 	Loss is: 0.6890964532470704 	Accuray: 56
Phase: validation 	Loss is: 0.6915320790863038 	Accuray: 55
Phase: training 	Loss is: 0.6767106209564209 	Accuray: 57
Phase: validation 	Loss is: 0.6836703589630126 	Accuray: 56
Phase: training 	Loss is: 0.6633904511260986 	Accuray: 60
Phase: validation 	Loss is: 0.6755468628692627 	Accuray: 58
Phase: training 	Loss is: 0.6486277323913574 	Accuray: 62
Phase: validation 	Loss is: 0.6698261340332031 	Accuray: 59
Phase: training 	Loss is: 0.6313288395690918 	Accuray: 64
Phase: validation 	Loss is: 0.6583011404418946 	Accuray: 61
Phase: training 	Loss is: 0.6123784197235107 	Accuray: 66
Phase: validation 	Loss is: 0.6486508506774903 	Accuray: 62
Phase: training 	Loss is: 0.5923671392822266 	Accuray: 6